In [1]:
from datetime import datetime

StatementMeta(, 66427f83-70c9-4149-bad8-6fb1f72b5c89, 3, Finished, Available, Finished)

In [2]:
today_date=datetime.today().strftime('%Y-%m-%d')


StatementMeta(, 66427f83-70c9-4149-bad8-6fb1f72b5c89, 4, Finished, Available, Finished)

In [2]:
Fabric_bronze_path='abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Tables/sales_silver'
from pyspark.sql.functions import col, to_date, lit
df=spark.read.format('delta').load(Fabric_bronze_path).filter(col('Processing_date')==to_date(lit(today_date)))

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 4, Finished, Available, Finished)

In [3]:
display(df)

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b816d640-5af8-4383-ac8e-71336fb411d3)

In [4]:
df.printSchema()

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 6, Finished, Available, Finished)

root
 |-- Row_ID: string (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Ship_Date: date (nullable = true)
 |-- Ship_Mode: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Postal_Code: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Market: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Shipping_Cost: double (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Month: string (nu

### creating dimension table
##### Dim_customer table

In [5]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, DateType
from delta.tables import DeltaTable
dim_customer_schema=StructType([
    StructField('Customer_ID',StringType(),True),
    StructField('Customer_Name',StringType(),True),
    StructField('Segment',StringType(),True),
    StructField('Postal_Code',StringType(),True),
    StructField('City',StringType(),True),
    StructField('State',StringType(),True),
    StructField('Country',StringType(),True),
    StructField('Region',StringType(),True),
    StructField('Market',StringType(),True),
])

DeltaTable.createIfNotExists(spark).tableName('Dim_Customer').addColumns(dim_customer_schema).execute()

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 7, Finished, Available, Finished)

##### Dim Product

In [6]:
dim_product_schema=StructType([
    StructField('Product_ID',StringType(),True),
    StructField('Category',StringType(),True),
    StructField('Sub_Category',StringType(),True),
    StructField('Product_Name',StringType(),True),
])

DeltaTable.createIfNotExists(spark).tableName('Dim_Product').addColumns(dim_product_schema).execute()

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 8, Finished, Available, Finished)

##### Fact sales table

In [7]:
Fact_Sales_schema=StructType([
    StructField('Row_ID',StringType(),True),
    StructField('Order_ID',StringType(),True),
    StructField('Customer_ID',StringType(),True),
    StructField('Product_ID',StringType(),True),
    StructField('Order_Date',DateType(),True),
    StructField('Ship_Date',DateType(),True),
    StructField('Ship_Mode',StringType(),True),
    StructField('Sales',DoubleType(),True),
    StructField('Quantity',IntegerType(),True),
    StructField('Discount',DoubleType(),True),
    StructField('Profit',DoubleType(),True),
    StructField('Shipping_Cost',DoubleType(),True),
    StructField('Order_Priority',StringType(),True),
    StructField('Month',StringType(),True),
    StructField('Year',StringType(),True),
    StructField('Processing_date',StringType(),True),
    StructField('Delivery_Days',StringType(),True),
    StructField('Profit_Margin',StringType(),True),
    ])

DeltaTable.createIfNotExists(spark).tableName('Fact_Sales').addColumns(Fact_Sales_schema).execute()

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 9, Finished, Available, Finished)

```
### loding data into tables
```
##### loading dim_customer data

In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

w = Window.partitionBy("Customer_ID").orderBy(col("Processing_date").desc())

df_selected_dim_customer = (
    df.select(
        "Customer_ID",
        "Customer_Name",
        "Segment",
        "Postal_Code",
        "City",
        "State",
        "Country",
        "Region",
        "Market",
        "Processing_date"
    )
    .withColumn("rn", row_number().over(w))
    .filter(col("rn") == 1)
    .drop("rn","Processing_date")
)


StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 10, Finished, Available, Finished)

In [9]:
from delta.tables import *
dim_customer_table_path='abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Tables/dim_customer'
dim_deltacustomer=DeltaTable.forPath(spark,dim_customer_table_path)
# perform the merge (upsert)
dim_deltacustomer.alias('target').merge(
    df_selected_dim_customer.alias('source'),'target.Customer_ID=source.Customer_ID'
    ).whenMatchedUpdate(set={
        'Customer_Name':'source.Customer_Name',
        'Segment':'source.Segment',
        'Postal_Code':'source.Postal_Code',
        'City':'source.City',
        'State':'source.State',
        'Country':'source.Country',
        'Region':'source.Region',
        'Market':'source.Market'
    }).whenNotMatchedInsert(values={
        'Customer_ID':'source.Customer_ID',
        'Customer_Name':'source.Customer_Name',
        'Segment':'source.Segment',
        'Postal_Code':'source.Postal_Code',
        'City':'source.City',
        'State':'source.State',
        'Country':'source.Country',
        'Region':'source.Region',
        'Market':'source.Market'
    }).execute()

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 11, Finished, Available, Finished)

In [10]:
history_df=dim_deltacustomer.history(1)
operation_metrics=history_df.select("operationMetrics").collect()[0][0]
rows_inserted=operation_metrics.get('numTargetRowsInserted',0)
rows_updated=operation_metrics.get('numTargetRowsUpdated',0)
rows_deleted=operation_metrics.get('numTargetRowsDeleted',0)
rows_affected=int(rows_inserted)+int(rows_updated)+int(rows_deleted)

print('Total rows of table:', dim_deltacustomer.toDF().count())
print(f"Rows inserted:{rows_inserted}")
print(f"Rows updated:{rows_updated}")
print(f"Rows deleted:{rows_deleted}")
print(f"Total rows affected:{rows_affected}")

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 12, Finished, Available, Finished)

Total rows of table: 849
Rows inserted:0
Rows updated:849
Rows deleted:0
Total rows affected:849


##### loding dim_product data

In [11]:
w = Window.partitionBy("Product_ID").orderBy(col("Processing_date").desc())

df_selected_dim_product = (
    df.select("Product_ID",
        "Category",
        "Sub_Category",
        "Product_Name",
        "Processing_date")
    .withColumn("rn", row_number().over(w))
    .filter(col("rn") == 1)
    .drop("rn","Processing_date")
)



StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 13, Finished, Available, Finished)

In [12]:
from delta.tables import *
dim_product_table_path='abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Tables/dim_product'
dim_deltaproduct=DeltaTable.forPath(spark,dim_product_table_path)
# perform the merge (upsert)
dim_deltaproduct.alias('target').merge(
    df_selected_dim_product.alias('source'),'target.Product_ID=source.Product_ID'
    ).whenMatchedUpdate(set={
        'Category':'source.Category',
        'Sub_Category':'source.Sub_Category',
        'Product_Name':'source.Product_Name'
    }).whenNotMatchedInsert(values={
        'Product_ID':'source.Product_ID',
        'Category':'source.Category',
        'Sub_Category':'source.Sub_Category',
        'Product_Name':'source.Product_Name'
    }).execute()

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 14, Finished, Available, Finished)

In [13]:
history_df=dim_deltaproduct.history(1)
operation_metrics=history_df.select("operationMetrics").collect()[0][0]
rows_inserted=operation_metrics.get('numTargetRowsInserted',0)
rows_updated=operation_metrics.get('numTargetRowsUpdated',0)
rows_deleted=operation_metrics.get('numTargetRowsDeleted',0)
rows_affected=int(rows_inserted)+int(rows_updated)+int(rows_deleted)

print('Total rows of table:', dim_deltaproduct.toDF().count())
print(f"Rows inserted:{rows_inserted}")
print(f"Rows updated:{rows_updated}")
print(f"Rows deleted:{rows_deleted}")
print(f"Total rows affected:{rows_affected}")

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 15, Finished, Available, Finished)

Total rows of table: 1185
Rows inserted:0
Rows updated:691
Rows deleted:0
Total rows affected:691


##### loading fact_sales data

In [14]:
df_selected_fact_sales = df.dropDuplicates(["Row_ID"])


StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 16, Finished, Available, Finished)

In [15]:
from delta.tables import *
fact_sales_table_path='abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Tables/fact_sales'
fact_deltasales=DeltaTable.forPath(spark,fact_sales_table_path)


# perform the merge (upsert)
fact_deltasales.alias('target').merge(
    df_selected_fact_sales.alias('source'),
    'target.Row_ID = source.Row_ID'
).whenMatchedUpdate(set={
        'Customer_ID':'source.Customer_ID',
        'Product_ID':'source.Product_ID',
        'Order_Date':'source.Order_Date',
        'Ship_Date':'source.Ship_Date',
        'Ship_Mode':'source.Ship_Mode',
        'Sales':'source.Sales',
        'Quantity':'source.Quantity',
        'Discount':'source.Discount',
        'Profit':'source.Profit',
        'Shipping_Cost':'source.Shipping_Cost',
        'Order_Priority':'source.Order_Priority',
        'Month':'source.Month',
        'Year':'source.Year',
        'Processing_date':'source.Processing_date',
        'Delivery_Days':'source.Delivery_Days',
        'Profit_Margin':'source.Profit_Margin'
}).whenNotMatchedInsert(values={
        'Row_ID':'source.Row_ID',
        'Order_ID':'source.Order_ID',
        'Customer_ID':'source.Customer_ID',
        'Product_ID':'source.Product_ID',
        'Order_Date':'source.Order_Date',
        'Ship_Date':'source.Ship_Date',
        'Ship_Mode':'source.Ship_Mode',
        'Sales':'source.Sales',
        'Quantity':'source.Quantity',
        'Discount':'source.Discount',
        'Profit':'source.Profit',
        'Shipping_Cost':'source.Shipping_Cost',
        'Order_Priority':'source.Order_Priority',
        'Month':'source.Month',
        'Year':'source.Year',
        'Processing_date':'source.Processing_date',
        'Delivery_Days':'source.Delivery_Days',
        'Profit_Margin':'source.Profit_Margin'
}).execute()


StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 17, Finished, Available, Finished)

In [16]:
history_df=fact_deltasales.history(1)
operation_metrics=history_df.select("operationMetrics").collect()[0][0]
rows_inserted=operation_metrics.get('numTargetRowsInserted',0)
rows_updated=operation_metrics.get('numTargetRowsUpdated',0)
rows_deleted=operation_metrics.get('numTargetRowsDeleted',0)
rows_affected=int(rows_inserted)+int(rows_updated)+int(rows_deleted)

print('Total rows of table:', fact_deltasales.toDF().count())
print(f"Rows inserted:{rows_inserted}")
print(f"Rows updated:{rows_updated}")
print(f"Rows deleted:{rows_deleted}")
print(f"Total rows affected:{rows_affected}")

StatementMeta(, 43f5a18b-7957-434c-9ef3-f69b02adeb6b, 18, Finished, Available, Finished)

Total rows of table: 3350
Rows inserted:0
Rows updated:1696
Rows deleted:0
Total rows affected:1696
